In [153]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os, sys, glob
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, box
import seaborn as sn
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
import xarray as xr

In [2]:
shp = gpd.read_file(r'C:\Doutorado\BD\IBGE\IBGE_Estruturas_cartograficas_Brasil\2017\Unidades_Censitarias\Municipios\MUNICIPIOS_PARA.shp')

shp.head()

,NM_MUNICIP,CD_GEOCMU,UF_ID,geometry
0,URUARÃ,1508159,15,POLYGON ((-53.82886240600001 -2.96325962599995...
1,REDENÃÃO,1506138,15,POLYGON ((-50.42375576800001 -7.74155583899990...
2,ABAETETUBA,1500107,15,POLYGON ((-48.83138539099997 -1.56352043699993...
3,BREJO GRANDE DO ARAGUAIA,1501758,15,POLYGON ((-48.30070702499997 -5.52300658699996...
4,FLORESTA DO ARAGUAIA,1503044,15,POLYGON ((-49.19127194999996 -7.14970454599997...


In [72]:
geofeature = shp.loc[shp['UF_ID'] == '15',:].dissolve(by='UF_ID')
geofeature.geometry = geofeature.geometry.convex_hull

geofeature.plot()

In [92]:

x, y = geofeature.geometry[0].exterior.coords.xy

new_pol = Polygon([ [xi, yi] for xi, yi in zip(x,y)])

darray_x = xr.DataArray(x, dims='point')
darray_y = xr.DataArray(y, dims='point')


In [87]:
ds = xr.tutorial.open_dataset('rasm').load()
ds

<xarray.Dataset>
Dimensions:  (time: 36, x: 275, y: 205)
Coordinates:
  * time     (time) object 1980-09-16 12:00:00 ... 1983-08-17 00:00:00
    xc       (y, x) float64 189.2 189.4 189.6 189.7 ... 17.65 17.4 17.15 16.91
    yc       (y, x) float64 16.53 16.78 17.02 17.27 ... 28.26 28.01 27.76 27.51
Dimensions without coordinates: x, y
Data variables:
    Tair     (time, y, x) float64 nan nan nan nan nan ... 29.8 28.66 28.19 28.21
Attributes:
    title:                     /workspace/jhamman/processed/R1002RBRxaaa01a/l...
    institution:               U.W.
    source:                    RACM R1002RBRxaaa01a
    output_frequency:          daily
    output_mode:               averaged
    convention:                CF-1.4
    references:                Based on the initial model of Liang et al., 19...
    comment:                   Output from the Variable Infiltration Capacity...
    nco_openmp_thread_number:  1
    NCO:                       "4.6.0"
    history:                   Tue D

In [88]:
%matplotlib

import cartopy.crs as ccrs

Using matplotlib backend: Qt5Agg


In [89]:
projection = ccrs.PlateCarree()

fig, ax = plt.subplots(1,1, sharex=True, sharey=True, subplot_kw={'projection':projection})

da = ds.isel(time=0)['Tair']

da.plot(ax=ax, x='x', y='y', transform=ccrs.PlateCarree(-180))
ax.gridlines(draw_labels=True,  transform=ccrs.PlateCarree())
plt.tight_layout()
plt.show()

In [7]:
fig2, ax = plt.subplots(subplot_kw={'projection':projection})

da2 = da.assign_coords(y=(((da.y + 90) % 180) - 90)).assign_coords(x=(((da.x + 180) % 360) - 180)).sortby(['x', 'y'])
da2.plot(ax=ax, transform=projection)
ax.gridlines(draw_labels=True,  transform=projection)
fig2.show()

In [8]:
import shapely.vectorized
from shapely.geometry import shape
import time

In [233]:


def xarray_mask(da, geom, lon='x', lat='y'):

    x0,x1 = geom.bounds[0], geom.bounds[2]
    y0,y1 = geom.bounds[1], geom.bounds[3]

    da_sel = da.sel({lon:slice(int(x0), int(x1)), lat:slice(int(y0), int(y1))})

    xx,yy = np.meshgrid(da_sel.coords['x'],da_sel.coords['y'])
    
    mask = shapely.vectorized.contains(geom,  xx,yy)
        
    return da_sel.where(mask, drop=False)

def _spatial_reduction(da_masked, reductions = [np.nanmean, np.nanmin, np.nanmax]):
    results = {}
    
    for func in reductions:
        try:
            temp = da_masked.reduce(func).values
            if np.isnan(temp):
                temp = 0
        except:
            temp = 0
        results[str(func.__name__)] = temp
            
    return results
    

def spatial_reduction(da, geom, lon='x', lat='y', reductions = [np.nanmean, np.nanmean, np.nanmax]):    
    masked_xarray = xarray_mask(da, geom, lon=lon, lat=lat)
    
    return _spatial_reduction(masked_xarray, reductions = reductions)

In [234]:
def spatial_reduction_shp_xarray(shp, dataArray, lon='x', lat='y', reductions = [np.nanmean, np.nanmean, np.nanmax]):

    shp2 = shp.apply(lambda feat: spatial_reduction(da, feat.geometry, lon, lat, reductions), axis=1)
    shp2 = shp2.apply(pd.Series)
    
    shp2['geometries'] = shp.geometry
    
    shp2.crs = shp.crs
    
    
    return shp2

In [235]:
spatial_reduction_shp_xarray(shp, da2)

C:\Anaconda3\lib\site-packages\xarray\core\variable.py:1464: RuntimeWarning: Mean of empty slice
  data = func(input_data, **kwargs)
C:\Anaconda3\lib\site-packages\xarray\core\variable.py:1464: RuntimeWarning: All-NaN slice encountered
  data = func(input_data, **kwargs)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,nanmean,nanmax,geometries
0,0,0,POLYGON ((-53.82886240600001 -2.96325962599995...
1,0,0,POLYGON ((-50.42375576800001 -7.74155583899990...
2,0,0,POLYGON ((-48.83138539099997 -1.56352043699993...
3,0,0,POLYGON ((-48.30070702499997 -5.52300658699996...
4,0,0,POLYGON ((-49.19127194999996 -7.14970454599997...
5,0,0,POLYGON ((-48.92560995699998 -3.41294027499992...
6,0,0,POLYGON ((-49.88005080299998 -1.11981425100001...
7,0,0,POLYGON ((-50.61493593799996 -0.65631342599990...
8,0,0,POLYGON ((-48.56309859599997 -4.82659764899993...
9,0,0,POLYGON ((-50.91908382999998 -5.15067194299994...


In [236]:
# option B:

In [239]:
import rasterio

def get_affine_from_xarray(da):
    
    '''
    a = width of a pixel
    b = row rotation (typically zero)
    c = x-coordinate of the upper-left corner of the upper-left pixel
    d = column rotation (typically zero)
    e = height of a pixel (typically negative)
    f = y-coordinate of the of the upper-left corner of the upper-left pixel
    '''
    dx = da.coords['x'][1] - da.coords['x'][0]
    dy = da.coords['y'][1] - da.coords['y'][0]
    newaff = rasterio.Affine(dx, 0, da.coords['x'].min(),
                             0, -dy, da.coords['y'][1].max())
    
    return newaff
    

In [238]:

newaff = get_affine_from_xarray(da)    

rasterized = rasterio.features.rasterize(geofeature, 
                                         out_shape=(da.coords['y'].size, da.coords['x'].size), 
                                         fill=1, out=None, 
                                         transform=newaff, 
                                         all_touched=False, 
                                         default_value=1, dtype=bool)
    

ModuleNotFoundError: No module named 'rasterio'